In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install autokeras

     |████████████████████████████████| 174kB 5.1MB/s 
     |████████████████████████████████| 71kB 4.7MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=be0a58a9a08b861591bca7739f385fa81d29962bed9848434bd8c05c98b9258b
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=82619f913f8d8197cd98688b6aaf5618f39eb4475da9f9af444a5bfd1b61b8e7
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df2= pd.read_csv("drugsComTest_raw.tsv", delimiter="\t")
df= pd.read_csv("drugsComTrain_raw.tsv",delimiter="\t")
df.rename(columns={'Unnamed: 0': 'UniqueId'}, inplace=True)
df2.rename(columns={'Unnamed: 0': 'UniqueId'}, inplace=True)

plt.hist(df['rating'])
plt.xlabel("Ratings")
plt.ylabel("No. of Reviews")
plt.title("Ratings vs No. of Reviews")
plt.show()


df= df.drop(['UniqueId', 'drugName', 'condition', 'date', 'usefulCount'], axis=1)

df2= df2.drop(['UniqueId', 'drugName', 'condition', 'date', 'usefulCount'], axis=1)

def sentiment(df):
    df.loc[df.rating<5, 'Sentiment']= 0
    df.loc[df.rating>=5, 'Sentiment']= 1
    return df


df = sentiment(df)
df= df.drop(['rating'], axis=1)

<Figure size 640x480 with 1 Axes>

In [2]:
import string
import re
from sklearn.model_selection import train_test_split

In [3]:
X = []
reviews = list(df['review'].astype(str))
for sen in reviews:
    X.append(sen)

In [4]:
X[0]

'"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"'

In [5]:
y = np.array(df['Sentiment'])

In [6]:
y

array([1., 1., 1., ..., 0., 1., 1.])

In [7]:
X_train, X_test,Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state = 45)

In [8]:
X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)
X_test = np.asarray(X_test)
Y_test = np.asarray(Y_test)

In [9]:
y = np.asarray(y)
print('Shape of data tensor:', X_train.shape)
print('Shape of data tensor:', X_test.shape)
print('Shape of label tensor:', y.shape)

Shape of data tensor: (129037,)
Shape of data tensor: (32260,)
Shape of label tensor: (161297,)


In [10]:
print('Shape of data tensor:', X_train.shape)
print('Shape of data tensor:', X_test.shape)
print('Shape of data tensor:', Y_train.shape)
print('Shape of data tensor:', Y_test.shape)

Shape of data tensor: (129037,)
Shape of data tensor: (32260,)
Shape of data tensor: (129037,)
Shape of data tensor: (32260,)


### Setting Up the Auto Keras Structured Data classifier model

In [11]:
import autokeras as ak

clf = ak.TextClassifier(
    overwrite=True, max_trials=1
)  # It only tries 1 model as a quick demo.
# Feed the text classifier with training data.
clf.fit(X_train, Y_train, epochs=3)
# Predict with the best model.
predicted_y = clf.predict(X_test)
# Evaluate the best model with testing data.
print(clf.evaluate(X_test, Y_test))

Trial 1 Complete [00h 31m 49s]
val_loss: 0.2674553692340851

Best val_loss So Far: 0.2674553692340851
Total elapsed time: 00h 31m 49s
INFO:tensorflow:Oracle triggered exit
Epoch 1/3
4033/4033 [==============================] - 735s 182ms/step - loss: 0.4081 - accuracy: 0.8172
Epoch 2/3
4033/4033 [==============================] - 808s 200ms/step - loss: 0.2786 - accuracy: 0.8832
Epoch 3/3
4033/4033 [==============================] - 683s 169ms/step - loss: 0.2299 - accuracy: 0.9050
INFO:tensorflow:Assets written to: .\text_classifier\best_model\assets
1009/1009 [==============================] - 47s 46ms/step - loss: 0.2699 - accuracy: 0.89290s - loss: 0.2699 - accuracy: 0.89
[0.2699384093284607, 0.8928704261779785]


In [12]:
n = np.random.randint(0,100)

X_test[n]

'"Hi.i was addicted to heroin for 9 years.a year ago decide I had enough so started off with methadone. Slowly I reduced it to 15ml daily and then started subtex.6mg fail. after only 4months I tempered down to 0.4. that is when they decided I was ready to stop and I happily agreed cos according to my doc such small does wouldn&#039;t effect me. The first day after taking last tablet I was ok.  But the next day I started feeling awful. I went back and explained that I had withdrawal symptoms. They told me I could go back on script but have to wait 10 days to see a doc (ridiculous) So back on heroin to keep me going till then.  Of course  I&#039;m only smoking really small amount. Worried tho after 10 days I&#039;ll need methadone again.  Does anyone know if I can go straight back on subtex?"'

In [13]:
if predicted_y[n] > 0.5:
  print('predicted sentiment : positive')
else: 
  print('precicted sentiment : negative')

if (Y_test[n] == 1):
  print('correct sentiment : positive')
else:
  print('correct sentiment : negative')

predicted sentiment : positive
correct sentiment : positive


In [14]:
predicted_y[n]

array([1.], dtype=float32)

In [15]:
X= np.array(X)
reviews_list_idx = X

In [16]:
def add_score_predictions(df, reviews_list_idx):

  df['sentiment score'] = 0


  review_preds = clf.predict(reviews_list_idx)

  df['sentiment score'] = review_preds

  pred_sentiment = np.array(list(map(lambda x : 'positive' if x > 0.5 else 'negative',review_preds)))

  df['predicted sentiment'] = 0

  df['predicted sentiment'] = pred_sentiment

  return df

In [17]:
df = add_score_predictions(df, reviews_list_idx)

In [18]:
df.head(6)

,review,Sentiment,sentiment score,predicted sentiment
0,"""It has no side effect, I take it in combinati...",1.0,1.0,positive
1,"""My son is halfway through his fourth week of ...",1.0,1.0,positive
2,"""I used to take another oral contraceptive, wh...",1.0,1.0,positive
3,"""This is my first time using any form of birth...",1.0,1.0,positive
4,"""Suboxone has completely turned my life around...",1.0,1.0,positive
5,"""2nd day on 5mg started to work with rock hard...",0.0,0.0,negative


In [20]:
df.to_csv('Drugs_Sentiments.csv', index=False)

In [21]:
x_review = []
reviews = list(df2['review'])
for sen in reviews:
    x_review.append(sen)

In [22]:
reviews_list_idx = np.array(x_review)

In [23]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53766 entries, 0 to 53765
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   review  53766 non-null  object 
 1   rating  53766 non-null  float64
dtypes: float64(1), object(1)
memory usage: 840.2+ KB


# Making Prediction on Test Dataset

In [24]:
def example(df2, reviews_list_idx):

  df2['sentiment score'] = 0

  #reviews_list_idx = pad_sequences(reviews_list_idx, maxlen=max_len, padding='post')
  
  review_preds = clf.predict(reviews_list_idx)

  df2['sentiment score'] = review_preds

  pred_sentiment = np.array(list(map(lambda x : 'positive' if x > 0.5 else 'negative',review_preds)))

  df2['predicted sentiment'] = 0

  df2['predicted sentiment'] = pred_sentiment

  return df2

In [33]:
df2.head(10)

,review,rating,sentiment score,predicted sentiment
0,"""I&#039;ve tried a few antidepressants over th...",10.0,1.0,positive
1,"""My son has Crohn&#039;s disease and has done ...",8.0,1.0,positive
2,"""Quick reduction of symptoms""",9.0,1.0,positive
3,"""Contrave combines drugs that were used for al...",9.0,1.0,positive
4,"""I have been on this birth control for one cyc...",9.0,1.0,positive
5,"""4 days in on first 2 weeks. Using on arms an...",4.0,1.0,positive
6,"""I&#039;ve had the copper coil for about 3 mon...",6.0,1.0,positive
7,"""This has been great for me. I&#039;ve been on...",9.0,1.0,positive
8,"""Ive been on Methadone for over ten years and ...",7.0,1.0,positive
9,"""I was on this pill for almost two years. It d...",2.0,0.0,negative


In [26]:
df2 = example(df2, reviews_list_idx)

In [32]:
df2

,review,rating,sentiment score,predicted sentiment
0,"""I&#039;ve tried a few antidepressants over th...",10.0,1.0,positive
1,"""My son has Crohn&#039;s disease and has done ...",8.0,1.0,positive
2,"""Quick reduction of symptoms""",9.0,1.0,positive
3,"""Contrave combines drugs that were used for al...",9.0,1.0,positive
4,"""I have been on this birth control for one cyc...",9.0,1.0,positive
...,...,...,...,...
53761,"""I have taken Tamoxifen for 5 years. Side effe...",10.0,0.0,negative
53762,"""I&#039;ve been taking Lexapro (escitaploprgra...",9.0,1.0,positive
53763,"""I&#039;m married, 34 years old and I have no ...",8.0,1.0,positive
53764,"""I was prescribed Nucynta for severe neck/shou...",1.0,0.0,negative
